Context

1. Project background：
This project is part of a big new program titled ‘A new Green Revolution: Manipulating the soil microbiome to enhance the sustainability of 21st century agriculture,’. The current focus is to optimise/engineer soil microbiomes to suppress the fungus Gaeumannomyces tritici, which causes the take-all disease in wheat (T. aestivum).

2. Microbial community coalescence：
It is the process by which two distinct microbial communities encounter one another and interact to form a new ‘daughter’ community. This assemblage may more closely resemble one ‘parent’ community than the other, and its functional profile may be totally unique. We plan to use a coalescence method to create new community to fight against take-all disease.


Obectives

Can we predict winners and losers during coalescence?
This very basic question remains relatively un-explored. However, we do had some insights on community assembly. That is, species with higher CUE value tend to survive. And there is certain connection between species richness and CUE variance.

We will use the similiar methdology as assembly to simulate the community dynamics during coalescence, mediated by temperature.

Microbial Consumer Resource Model

$$
\frac{dC_i}{dt} = \sum_{\alpha=0}^{M} C_i R_{\alpha} u_{i\alpha} (1 - \lambda_{\alpha}) - C_i m_i
$$

$$
\frac{dR_{\alpha}}{dt} = \rho_{\alpha} - R_{\alpha} \omega_{\alpha} - \sum_{i=0}^{N} C_i R_{\alpha} u_{i\alpha} + \sum_{i=0}^{N} \sum_{\beta=0}^{M} C_i R_{\beta} u_{i\beta} l_{\beta\alpha}
$$


| **Parameter** | **Description** | **Key** |
|--------------|---------------|--------|
| $C_i$ | Biomass of the $i$th consumer | - |
| $R_{\alpha}$ | Mass of the $\alpha$th resource | - |
| $N$ | Number of consumer populations | $N$ |
| $M$ | Number of resources | $M$ |
| $u_{i\alpha}$ | Uptake rate of the $\alpha$th resource by the $i$th consumer | $u$ |
| $m_i$ | Loss term for the $i$th consumer | $m$ |
| $\rho_{\alpha}$ | Inflow rate for the $\alpha$th resource | $\rho$ |
| $\omega_{\alpha}$ | Outflow term for the $j$th resource | $\omega$ |
| $l_{\alpha\beta}$ | Proportion of uptake of the $\alpha$th resource leaked to the $\beta$th resource. | $l$ |
| $\lambda_{\alpha}$ | Total proportion of the $\alpha$th resource leaked, same as $\sum_{\beta} l_{\alpha\beta}$ | $\lambda$ |


inputs and outputs
why do we use this method

Simulation:

Import the packages

In [ ]:
   using Pkg
   Pkg.activate("../packages")
   using Distributions
   using MiCRM

Set system size and leakage rate

In [ ]:
   N,M,leakage = 100,50,0.3

N is the number of species in a community, M is the number of resource types, leakage is the overall leakage rate

Make uptake matrix out of dirichlet distribution

In [ ]:
   u =  MiCRM.Parameters.modular_uptake(M,N; N_modules = 2, s_ratio = 10.0)

Define other terms

In [ ]:
   #cost term
   m = ones(N)

   #resource inflow + outflow
   ρ,ω = ones(M),ones(M)

m is a loss term for the the consumers; ρ and ω are inflow rate and outflow rate for the resource respectively.

Make leakage matrix out of dirichlet distribution and make it modular

In [ ]:
   l = MiCRM.Parameters.modular_leakage(M; N_modules = 2, s_ratio = 10.0, λ = leakage)

Set original state of the system and solve it

In [ ]:
   #define parameters
   param = MiCRM.Parameters.generate_params(N, M;  u = u, m = m, ρ = ρ, ω = ω, l = l, λ = leakage)
   
   #original state
   x0 = ones(N+M)
   #time span
   tspan = (0.0, 10.0)

   #define problem
   using DifferentialEquations
   prob = ODEProblem(MiCRM.Simulations.dx!, x0, tspan, param)
   

create an ODEProblem object which defines the problem for the ODE solver and then solve it with the aptly named solve function.
To define the ODEProblem we need to specify the initial state of the system as well as the timespan

In [ ]:
   sol = solve(prob, Tsit5())

In this case, we use Tsit5 as a solver, which is widely applicable for non-stiff problems.
If need more information about the solver choice, refer to https://docs.sciml.ai/DiffEqDocs/stable/solvers/ode_solve/

Calculate key properties

In [ ]:
   # jacobian of system from the solution object
   J = MiCRM.Analysis.get_jac(sol)
   # generate purbulence matrix
   pur = rand(size(J, 1))
   # the instantaneous rate of growth of the perturbation pur at time t.
   t = 5
   MiCRM.Analysis.get_Rins(J, pur, t)
   # Determine the stability a system given its jaccobian by testing if the real part of the leading eigenvalue is positive.
   MiCRM.Analysis.get_stability(J)
   #test whether a system is reactive to the perturbation 
   MiCRM.Analysis.get_reactivity(J,pur)
   #get the rate of return of the system from perturbation
   MiCRM.Analysis.get_return_rate(J)

Visualization

figures

Outputs and next step

Outputs: two assembled stable community, with the information of each species' resource uptake rate and leakage 

Documents of functions

MiCRM.Parameters.modular_uptake  
```MiCRM.Parameters.modular_uptake(M,N; N_modules = 2, s_ratio = 10.0)```  
It helps generate an uptake matrix using a Dirichlet distribution such that the uptake of all consumers sums to 1.
The number of modules determines how many groups of resources the consumers are specialised over. For example if N_modules = 2 then the resources will be split into two groups with half the consumers specialising on one and half on the other.

The degree of specialisation is determined by the s_ratio value. This controls the relative value of the dirchlet α parameters which determine how the probabiltiy density function is distributed over the different resources. Specialisation is obtained by setting the α values for resources that specialists consume to higher values meaning they have a higher probablity of a larger value. When s_ratio = 1 the proabailtiy is uniform and all resources are equally likely to be consumed. When s_ratio >1 then consumers are more likely to consume resources within thier module.

MiCRM.Parameters.modular_leakage  
``` MiCRM.Parameters.modular_leakage(M; N_modules = 2, s_ratio = 10.0, λ = leakage)```  
It generates a modular leakage matrix with a directional leakage structure. The matrix is generated using a Dirichlet distribution such that the leakage of all resources sums to λ.

The number of modules determines how many groups of resources they are split into. For example if N_modules = 5 then the resources will be split into five groups with the first group of resources tending to leak to the second, the second to the third and so on.

The degree of to which resources leak in this constrained way (verses randomly across all resources) is determined by the s_ratio value. This controls the relative value of the dirchlet α parameters which determine how the probabiltiy density function is distributed over the different resources. A greater probabiltiy of leakage from one resource to another is obtained by setting the α values for resources to higher values. When s_ratio = 1 the proabailtiy is uniform and all resources are equally likely to be leaked to each other. When s_ratio >1 then resources are more likely to leak to resources thier own or the next module in the sequence.

MiCRM.Parameters.generate_params  
```MiCRM.Parameters.generate_params(N, M;  u = u, m = m, ρ = ρ, ω = ω, l = l, λ = leakage)```  
It generates parameter sets for MiCRM simualtions. Requires the system size is defined as well as the functions to generate the actual parameters used in the simulations. Extra arguments can also be passed via the kwargs argument.By default the function generates a random MiCRM system with no structure and uptake and leakage matricies generated by Dirichlet distribitions.

The function returns a NamedTuple with all the parameters as well as a kw entry which itself is a NamedTuple with the additional arguments provided.